In [1]:
def fxses(date, security):
    x = dt.datetime.timestamp(dt.datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S'))
    element = dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d')
    y = dt.datetime.strptime(element, '%Y-%m-%d')
    y_ts = dt.datetime.timestamp(y)

    # above codes, capture and change current time to timestamp
    # Convert the timestamp into YYYY-MM-DD format, in a way stripping the HH:MM:SS elements
    # capture the timestamp of the stripped date data, it would arrive YYYY-MM-DD, 00:00:00
    
    if y.weekday() == 6: # if the date was a Sunday, refer 2 days back, which is Friday
        tokyo_0 = y_ts - 172800 - 3600
        tokyo_1 = y_ts - 172800 + 28800
        london_0 = y_ts - 172800 + 25200
        london_1 = y_ts - 172800 + 57600
        newyork_0 = y_ts - 172800 + 43200
        newyork_1 = y_ts - 172800 + 72000
    elif y.weekday() == 0: # if the date was a Monday, refer 3 days back, which is Friday
        tokyo_0 = y_ts - 259200 - 3600
        tokyo_1 = y_ts - 259200 + 28800
        london_0 = y_ts - 259200 + 25200
        london_1 = y_ts - 259200 + 57600
        newyork_0 = y_ts - 259200 + 43200
        newyork_1 = y_ts - 259200 + 72000
    else: 
        tokyo_0 = y_ts - 86400 - 3600
        tokyo_1 = y_ts - 86400 + 28800
        london_0 = y_ts - 86400 + 25200
        london_1 = y_ts - 86400 + 57600
        newyork_0 = y_ts - 86400 + 43200
        newyork_1 = y_ts - 86400 + 72000
        
    # above using timestamp to segment out the timestamp for the respective hours of the previous day
    # value of 1 represent 1 second, so convert to hours accordingly
    
    tokyo_b = dt.datetime.fromtimestamp(tokyo_0)
    tokyo_e = dt.datetime.fromtimestamp(tokyo_1)
    london_b = dt.datetime.fromtimestamp(london_0)
    london_e = dt.datetime.fromtimestamp(london_1)
    newyork_b = dt.datetime.fromtimestamp(newyork_0)
    newyork_e = dt.datetime.fromtimestamp(newyork_1)
    
    # above convert the timestamp to datetime value
    
    tokyo_high = markets.loc[tokyo_b:tokyo_e]['MID_HIGH'].max()
    tokyo_low = markets.loc[tokyo_b:tokyo_e]['MID_LOW'].min()
    london_high = markets.loc[london_b:london_e]['MID_HIGH'].max()
    london_low = markets.loc[london_b:london_e]['MID_LOW'].min()
    newyork_high = markets.loc[newyork_b:newyork_e]['MID_HIGH'].max()
    newyork_low = markets.loc[newyork_b:newyork_e]['MID_LOW'].min()
    
    # above using reuters API to capture the high & low in hourly interval
    
    if newyork_high > london_high > tokyo_high and newyork_low > london_low > tokyo_low:
        trade = True
        direction = 'long'
        return trade, direction, london_high, london_low
    elif newyork_high < london_high < tokyo_high and newyork_low < london_low < tokyo_low:
        trade = True
        direction = 'short'
        return trade, direction, london_high, london_low
    else: 
        trade = False
        direction = 'No'
        return trade, direction, london_high, london_low
    
    # above is the trading rule, the key in this whole exercise
    # target to return trade signal, if neither criteria fulfilled, return False
    # False indicating no trade
    

In [2]:
def sameday(i):
    
    # converting the timestamp to datetime object and extract the date values
    x = dt.datetime.strptime(str(markets.index[i-1]), '%Y-%m-%d %H:%M:%S').date()
    y = dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date()
    
    # return true if it is the same date
    return (x == y)

In [13]:
import refinitiv.data as rd
import datetime as dt
import pandas as pd

rd.open_session()
data = []

security = ['AUD=', 'NZD=', 'EUR=', 'GBP=', 'CAD=', 'JPY=', 'CHF=', 'EURGBP=', 'AUDNZD=']

date_0 = '2021-12-01'
date_1 = '2023-12-01'

for ccy in security:
    markets = rd.get_history(universe=[ccy], fields=['MID_HIGH','MID_LOW'],
                         interval="1h", start=date_0, end=date_1
                        )

    entry = 0
    SL = 0 
    TP = 0
    pnl = []

    for i in range(24, len(markets)): 
        try:
            if entry == 0: # if no trade
                trade, direction, high, low = fxses(markets.index[i], security)
                if sameday(i) == False:
                    if dt.datetime.strptime(str(markets.index[i]), '%Y-%m-%d %H:%M:%S').date().weekday() == 6: # a brute force way to resolve issue on weekend
                        continue
                    elif trade == False:
                        continue
                    elif direction == 'long':
                        entry = markets['MID_HIGH'][i]
                        SL = low
                        TP = entry + (entry - SL)
                        trade_entry[markets.index[i]] = entry
                    elif direction == 'short':
                        entry = -markets['MID_LOW'][i]
                        SL = high
                        TP = -entry + (-entry - SL)
                        trade_entry[markets.index[i]] = entry
                else: continue
            elif entry > 0: # if long
                if markets['MID_HIGH'][i] >= TP:
                    pnl.append(markets['MID_HIGH'][i] - entry)
                    entry = 0
                    SL = 0
                    TP = 0
                    continue
                elif markets['MID_LOW'][i] <= SL:
                    pnl.append(markets['MID_LOW'][i] - entry)
                    entry = 0
                    SL = 0
                    TP = 0
                    continue
                else: continue
            elif entry < 0: # if short
                if markets['MID_HIGH'][i] >= SL:
                    pnl.append(-entry - markets['MID_HIGH'][i])
                    entry = 0
                    SL = 0
                    TP = 0
                    continue
                elif markets['MID_LOW'][i] <= TP:
                    pnl.append(-entry - markets['MID_LOW'][i])
                    entry = 0
                    SL = 0
                    TP = 0
                    continue
                else: continue
            else: continue
        except:
            continue
            
    win_ratio = sum(x>0 for x in pnl)/len(pnl)
    reward_risk = sum(x>0 for x in pnl)/sum(x<0 for x in pnl)
    
    data.append([ccy, len(pnl), sum(pnl), win_ratio, reward_risk])

summary = pd.DataFrame(data, columns=['CCY pair', 'Number of trades', 'Total PnL', 'Win%', 'Win/Loss'])
display(summary)

,CCY pair,Number of trades,Total PnL,Win%,Win/Loss
0,AUD=,23,-0.02505,0.478261,0.916667
1,NZD=,26,-0.01340,0.384615,0.625000
2,EUR=,24,-0.07775,0.333333,0.500000
3,GBP=,25,-0.00625,0.440000,0.785714
4,CAD=,23,-0.04865,0.434783,0.769231
5,JPY=,40,-19.73500,0.425000,0.739130
6,CHF=,20,-0.07735,0.300000,0.428571
7,EURGBP=,17,0.02160,0.529412,1.125000
8,AUDNZD=,41,-0.03040,0.414634,0.708333
